In [ ]:
"""
Code for PR Loss and NCFPR
"""
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Optinal

In [ ]:
def bpr_loss(pos: torch.Tensor, neg: torch.Tensor)-> torch.Tensor:
    """Bayesian Personalized Ranking Loss

    Parameters
    ----------
    pos : torch.Tensor
        Ranking logit (0..1)
    neg : torch.Tensor
        Ranking logit (0..1)
    
    Return
    ------
    loss scalar
    """
    diff = pos - neg
    return -F.logsigmoid(diff).mean()

In [ ]:
def hinge_loss(pos: torch.Tensor, neg: torch.Tensor)-> torch.Tensor:
    diff = pos - neg
    return torch.maximum(margin-diff, 0).mean()

In [ ]:
class ML100kData(Dataset):
    def __init__(self, data_dir:str="../Data/ml-100k", normalize_rating:bool=False):
        self.data_dir = data_dir
        self.normalize_rating = normalize_rating
        self.df, self.num_users, self.num_items = read_data_ml100k(data_dir)
        self.user_id = self.df.user_id.values - 1
        self.item_id = self.df.item_id.values - 1
        self.rating = self.df.rating.values.astype(np.float32)
        
    def split(self, train_ratio=0.8):
        train_len = int(train_ratio * len(self))
        test_len = len(self) - train_len
        return random_split(self, [train_len, test_len])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx:int):
        return self.user_id[idx], self.item_id[idx], self.rating[idx]

class ML100KPairWise(ML100kData):
    def __init__(self, data_dir="../Data/ml-100k",
                 test_leave_out=1,
                 test_sample_size: int = None):
        """Pair Wise loader to train NeuMF model.
        Samples are slightly different based on train/test mode.

        In training mode:
        - user_id: int
        - item_id: int
            Item id that user has interacted with
        - neg_item: int
            Item id that user hasn't interacted with while training

        In testing mode:
        - user_id: int
        - item_id: int
            Random item_id to be ranked by the model
        - is_pos: bool
            If True, this item is a positive item 
            that user has interacted with in groundtruth data.


        Parameters
        ----------
        data_dir : str, optional
            Path to dataset directory, by default "./ml-100k"
        test_leave_out : int, optional
            Leave out how many items per user for testing
            By default 1
        test_sample_size : int, optional
            It is time-consuming to rank all items for every user during
            evaluation, we can randomly choose a subset of items to rank
            If None, rank all items.
            By default None
        """
        super().__init__(data_dir)
        # Use 0-based indexing consistently
        self.set_all_item_ids = set(range(self.num_items))  # 0 to num_items-1
        self.test_leave_out = test_leave_out
        self.test_sample_size = test_sample_size
        # general
        self.train = None
        self.has_setup = False
        # Split Dataframe
        self.split_dataframe()
        self.build_candidates()

    def split_dataframe(self):
        """Split ML100K dataframe with the strategy leave-n-out
        with timestamp order.
        """
        user_group = self.df.groupby("user_id", sort=False)
        train_df = []
        test_df = []
        for user_id, user_df in user_group:
            user_df = user_df.sort_values("timestamp")
            train_df.append(user_df[:-self.test_leave_out])
            test_df.append(user_df[-self.test_leave_out:])
        self.train_df = pd.concat(train_df)
        self.test_df = pd.concat(test_df)

    def build_candidates(self):
        # Train - Make both user_id and item_id 0-based
        self.observed_items_per_user_in_train = {
            int(user_id) - 1: user_df.item_id.values - 1
            for user_id, user_df in self.train_df.groupby("user_id", sort=False)
        }
        self.unobserved_items_per_user_in_train = {
            user_id: np.array(
                list(self.set_all_item_ids - set(observed_items)))
            for user_id, observed_items in self.observed_items_per_user_in_train.items()
        }
        # Test - Make both user_id and item_id 0-based
        self.gt_pos_items_per_user_in_test = {
            int(user_id) - 1: user_df[-self.test_leave_out:].item_id.values - 1
            for user_id, user_df in self.test_df.groupby("user_id", sort=False)
        }

    def split(self, *args, **kwargs):
        # Train split
        train_split = deepcopy(self)
        train_split.user_id = self.train_df.user_id.values - 1
        train_split.item_id = self.train_df.item_id.values - 1
        train_split.train = True
        train_split.has_setup = True
        
        # Test split
        test_split = deepcopy(self)
        test_split.user_id = []
        test_split.item_id = []
        for user_id, items in self.unobserved_items_per_user_in_train.items():
            if self.test_sample_size is None:
                sample_items = items
            elif isinstance(self.test_sample_size, int):
                sample_items = np.random.choice(items, self.test_sample_size)
            else:
                raise TypeError("self.test_sample_size should be int")
            sample_items = np.concatenate(
                [test_split.gt_pos_items_per_user_in_test[user_id],
                 sample_items])
            sample_items = np.unique(sample_items)
            test_split.user_id += [user_id]*len(sample_items)
            test_split.item_id.append(sample_items)
        test_split.user_id = np.array(test_split.user_id)
        test_split.item_id = np.concatenate(test_split.item_id)
        test_split.train = False
        test_split.has_setup = True
        return train_split, test_split

    def __len__(self):
        return len(self.user_id)

    def __getitem__(self, idx):
        assert self.has_setup, "Must run self.setup()"
        if self.train:
            user_id = self.user_id[idx]
            pos_item = self.item_id[idx]
            neg_item = np.random.choice(
                self.unobserved_items_per_user_in_train[int(user_id)])
            return user_id, pos_item, neg_item
        else:
            user_id = self.user_id[idx]
            item_id = self.item_id[idx]
            is_pos = item_id in self.gt_pos_items_per_user_in_test[user_id]
            return user_id, item_id, is_pos

class LitData(L.LightningDataModule):
    def __init__(
        self, 
        dataset:Dataset, 
        train_ratio:float=0.8, 
        batch_size:int=32, 
        num_workers:int=4
    ):
        self.dataset = dataset
        self.train_ratio = train_ratio
        self.dataloader_kwargs = {
            "batch_size": batch_size,
            "num_workers": num_workers,
            "persistent_workers": True if num_workers > 0 else False
        }
        self._log_hyperparams = True
        self.allow_zero_length_dataloader_with_multiple_devices = False

        self.num_users = getattr(self.dataset, "num_users", None)
        self.num_items = getattr(self.dataset, "num_items", None)

    def setup(self, stage:str):
        self.train_split, self.test_split = self.dataset.split(
            self.train_ratio)

    def train_dataloader(self):
        return DataLoader(self.train_split, **self.dataloader_kwargs, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_split, **self.dataloader_kwargs, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.test_split, **self.dataloader_kwargs, shuffle=False)

In [ ]:
class NeuMF(nn.Module):
    def __init__(self, embedding_dims: int, num_users: int, num_items: int, hidden_dims: List, **kwargs):
        super().__init__()
        self.P = nn.Embedding(num_users, embedding_dims)
        self.Q = nn.Embedding(num_items, embedding_dims)
        self.U = nn.Embedding(num_users, embedding_dims)
        self.V = nn.Embedding(num_items, embedding_dims)
        mlp = [nn.Linear(embedding_dims*2, hidden_dims[0]),
               nn.ReLU()]
        for i in range(len(hidden_dims) - 1):
            mlp += [nn.Linear(hidden_dims[i], hidden_dims[i+1]),
                    nn.ReLU()]
        self.mlp = nn.Sequential(*mlp)
        self.output_layer = nn.Linear(
            hidden_dims[-1] + embedding_dims, 1, bias=False)

    def forward(self, user_id, item_id) -> torch.Tensor:
        p_mf = self.P(user_id)
        q_mf = self.Q(item_id)
        gmf = p_mf * q_mf

        p_mlp = self.U(user_id)
        q_mlp = self.V(item_id)
        mlp = self.mlp(torch.cat([p_mlp, q_mlp], axis=-1))
        logit = self.output_layer(
            torch.cat([gmf, mlp], axis=-1))
        return logit

class LitNeuMF(L.LightningModule):
    def __init__(self, lr=0.002, hitrate_cutout=10, **kwargs):
        super().__init__()
        self.save_hyperparameters()
        self.model = NeuMF(**kwargs)
        self.lr = lr
        self.hitrate = RetrievalHitRate(top_k=hitrate_cutout)
        self.training_step_outputs = []

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.lr, weight_decay=1e-5)

    def forward(self, user_id, item_id):
        return self.model(user_id, item_id)

    def training_step(self, batch, batch_idx):
        user_id, pos_item, neg_item = batch
        pos_score = self(user_id, pos_item)
        neg_score = self(user_id, neg_item)
        loss = bpr_loss(pos_score, neg_score)
        self.training_step_outputs.append(loss)
        return loss

    def validation_step(self, batch, batch_idx):
        user_id, item_id, is_pos = batch
        logit = self(user_id, item_id)
        score = torch.sigmoid(logit).reshape(-1,)
        self.hitrate.update(score, is_pos, user_id.to(torch.int64))
        return 

    def on_train_epoch_end(self):
        epoch_average = torch.stack(self.training_step_outputs).mean()
        self.logger.experiment.add_scalar(
            "train/loss", epoch_average, self.current_epoch)
        self.training_step_outputs.clear()

    def on_validation_epoch_end(self):
        self.logger.experiment.add_scalar(
            f"val/hit_rate@{self.hitrate.top_k}", self.hitrate.compute(), self.current_epoch)
        self.hitrate.reset()

In [ ]:
def neural_mf():
    embedding_dims, max_epochs, batch_size = 30, 40, 256
    data = LitData(
        ML100KPairWise(test_sample_size=100),
        batch_size=batch_size,
        num_workers=4
    )
    
    model = LitNeuMF(
        num_users=data.num_users, num_items=data.num_items,
        embedding_dims=embedding_dims,
        hidden_dims=[10, 10, 10]
    )

    logger = TensorBoardLogger("log", name=f"NeuMF")
    trainer = L.Trainer(max_epochs=max_epochs, accelerator="auto", logger=logger)

    trainer.fit(model, data)